In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score

# Function to extract MFCC features and their deltas
def extract_mfcc_features(file_path, n_mfcc=13, delta_order=2):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_delta = librosa.feature.delta(mfcc, order=1)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    features = np.vstack([mfcc, mfcc_delta, mfcc_delta2])
    features = np.mean(features, axis=1)
    return features

# Function to extract Log Mel Spectrogram
def extract_mel_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return np.mean(log_mel_spectrogram, axis=1)

# Set file paths based on the Kaggle competition directory structure
train_file = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
test_file = '/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv'
train_audio_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train'
test_audio_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test'

# Load the CSV files
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Extract features from the audio files in the 'audios_train' folder (training data)
X_train = []
y_train = []

for idx, row in train_df.iterrows():
    file_path = os.path.join(train_audio_dir, row['filename'])
    if os.path.exists(file_path):  
        mfcc_features = extract_mfcc_features(file_path)
        mel_features = extract_mel_spectrogram(file_path)  
        X_train.append(np.concatenate([mfcc_features, mel_features]))  
        y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Scale the features to improve model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Hyperparameter tuning for Gradient Boosting Regressor
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
}

model = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_scaled, y_train)

best_model = grid_search.best_estimator_

# Cross-validation
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean Cross-Validation Score: {cv_scores.mean()}")

# Extract and scale test data
X_test = []

for filename in test_df['filename']:
    file_path = os.path.join(test_audio_dir, filename)
    if os.path.exists(file_path):
        mfcc_features = extract_mfcc_features(file_path)
        mel_features = extract_mel_spectrogram(file_path)
        X_test.append(np.concatenate([mfcc_features, mel_features]))

X_test = np.array(X_test)
X_test_scaled = scaler.transform(X_test)

# Predict on the test set using the best model
y_pred = best_model.predict(X_test_scaled)

# Prepare the submission file
submission = pd.DataFrame({'filename': test_df['filename'], 'label': y_pred})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("Submission file has been generated.")


Cross-Validation Scores: [-1.5514484  -0.69443373 -0.60522797 -0.80197674 -1.54687066]
Mean Cross-Validation Score: -1.0399915005508757
